In [ ]:
import anyio
from tqdm.notebook import tqdm
import asyncio

In [ ]:
# | notest

num_msgs = 10_000_000

send_pbar = tqdm(total=num_msgs)
receive_pbar = tqdm(total=num_msgs*2)

async def latency_task():
    receive_pbar.update(1)
    await asyncio.sleep(15)
    receive_pbar.update(1)

async def process_message_callback(
        receive_stream,
) -> None:
    async with receive_stream:
        async for msgs in receive_stream:
            pass
            #await msgs

send_stream, receive_stream = anyio.create_memory_object_stream(
    max_buffer_size=num_msgs
)

async with anyio.create_task_group() as tg:
    tg.start_soon(process_message_callback, receive_stream)
    async with send_stream:
        for i in range(num_msgs):
            await send_stream.send(latency_task())
            send_pbar.update(1)
            if i%1000 == 999:
                send_pbar.refresh()

In [ ]:
for i in tqdm(range(100)):
    pass

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
async def _streamed_records(
    receive_stream,
):
    async for records_per_topic in receive_stream:
        for records in records_per_topic:
            for record in records:
                yield record

In [ ]:
# | notest

num_msgs = 100_000

send_pbar = tqdm(total=num_msgs)
receive_pbar = tqdm(total=num_msgs*2)

async def process_message_callback(
        receive_stream,
) -> None:
    async with receive_stream:
        async for msgs in _streamed_records(receive_stream):
            receive_pbar.update(1)
            await asyncio.sleep(15)

send_stream, receive_stream = anyio.create_memory_object_stream(
    max_buffer_size=num_msgs
)

async with anyio.create_task_group() as tg:
    tg.start_soon(process_message_callback, receive_stream)
    async with send_stream:
        for i in range(num_msgs):
            await send_stream.send([["Hello"]*2])
            send_pbar.update(1)
            

In [ ]:
def callback(task):
    print(task)
#     print(task.exception())
#     task.result()
#     print("Done?")

async def throws_exception():
    raise RuntimeError("My runtime error!")
    
task = asyncio.create_task(throws_exception())
task.add_done_callback(callback)

### Task reference handling